<a href="https://colab.research.google.com/github/hjfu/SC_project/blob/master/Reserve_Forecast_ConsumerOther_2019Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import os
import pandas as pd
import dask.dataframe as dd
import numpy as np
import datetime as dt
import collections
from pandas.tseries.offsets import MonthEnd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm

#import self written module for vintage calculation and visualization
# sys.path.append('C:\\Users\\Hu09559\\Documents\\Consumer\\Code\\Python')
# import vintageCal

In [0]:
# dataDirectory = 'W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer_Models\\SAS_MIP\\ConsumerOther\\Data\\'
#dataDirectory = 'C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\'
# fileName = ''.join(['cecl_ptf_20190630_consumerother.sas7bdat'])
#fileName = ''.join('cecl_ptf_20190630_consumerother.csv')

dataDirectory = 'W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\data'
fileName = ''.join('CECL_PTF_20190930.csv')
filePath = os.path.join(dataDirectory, fileName)
data_snapshot = pd.read_csv(filePath)
print(fileName)
# data_snapshot =  pd.read_sas(filePath,format='sas7bdat')


CECL_PTF_20190930.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (42,78,82,137,143) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#print(data_snapshot.columns[data_snapshot.columns.str.contains('EOM')])
# print(data_snapshot.columns[data_snapshot.columns.str.contains('Open')])
#print(list(data_snapshot.columns))
print(data_snapshot.columns[data_snapshot.columns.str.contains('Seg')])
# print(data_snapshot.columns[data_snapshot.columns.str.contains('Due')])
print(data_snapshot['SECSegmentDe_YrMo'].unique())

Index(['SECSegmentDe_Origin', 'Model_Segment', 'SECSegmentNr_YrMo',
       'SECSegmentDe_YrMo', 'SECSubSegmentDe_YrMo'],
      dtype='object')
['Consumer Real Estate' 'Commercial and Industrial' 'Consumer and Other'
 'CRE - Owner Occupied' 'CRE - Non Owner Occupied' 'Energy'
 'CRE - Construction and Land Development' nan]


In [0]:
data_snapshot = data_snapshot.loc[data_snapshot['SECSegmentDe_YrMo'] == 'Consumer and Other',]

In [0]:
# define the default loans

default_df = data_snapshot.loc[((data_snapshot['DAYS_PAST_DUE_CNT'] >= 90) &(data_snapshot['DAYS_PAST_DUE_CNT'] < 120)) |(data_snapshot['EOMChrgOffDt'].notnull()) | (data_snapshot['RiskCd'] >=12), ] 

In [0]:
default_df.shape[0]/data_snapshot.shape[0]

0.0015384615384615385

In [0]:
data_snapshot.shape[0]

27950

In [0]:
print(data_snapshot['SECSegmentDe_YrMo'].unique())

['Consumer and Other']


Data Cleaning¶

In [0]:
# EOMDates = list(data_snapshot.columns[data_snapshot.columns.str.contains('YrMo')])
# df_EOMcols = data_snapshot[EOMDates]
# print(df_EOMcols)
# print(data_snapshot.EOMOpenDt_YrMo.head())
# def numConvert2Dates (x):
# #     x1 = x.astype(int)
#     x2 = x.astype(str)
#     x3 = x2[:6]
#     return (x3)

# data_snapshot[['EOMMaturityDt_YrMo', 'EOMOpenDt_YrMo','YrMo']].apply(numConvert2Dates)


#### Create a new column named "Date" from YrMo as Timestamp and period¶

In [0]:
data_snapshot['Date_dt'] = pd.to_datetime(data_snapshot['YrMo'].astype(str), format='%Y%m')
data_snapshot['Date_dt'] = data_snapshot['Date_dt'] + MonthEnd()
data_snapshot["Date_mon"] = data_snapshot["Date_dt"].dt.to_period("M")
print(data_snapshot.EOMMaturityDt_YrMo.head())

data_snapshot['MaturityDate_dt'] = pd.to_datetime(data_snapshot['EOMMaturityDt_YrMo'].astype(str), format='%d%b%Y')
# data_snapshot['MaturityDate_dt'] = data_snapshot['MaturityDate_dt'] + MonthEnd()
data_snapshot["MaturityDate_mon"] = data_snapshot["MaturityDate_dt"].dt.to_period("M")
data_snapshot['vintage_year'] = pd.to_datetime(data_snapshot['EOMOpenDt_YrMo'].astype(str), format='%d%b%Y').dt.strftime("%Y")


print(data_snapshot['MaturityDate_mon'].head())

2    31OCT2021
4    31MAR2020
6    31OCT2021
7    31DEC2019
9    31OCT2020
Name: EOMMaturityDt_YrMo, dtype: object
2   2021-10
4   2020-03
6   2021-10
7   2019-12
9   2020-10
Name: MaturityDate_mon, dtype: object


In [0]:
print(data_snapshot.loc[data_snapshot['INSTID'] == 1113180, 'MaturityDate_dt'])

Series([], Name: MaturityDate_dt, dtype: datetime64[ns])


#### Create two level segmentation variables (1): segment_level_0

In [0]:
# data_snapshot['segment_level_0'] = 5
# data_snapshot.loc[(data_snapshot.ProdCd_Origin == 'HEQ')|(data_snapshot.ProdCd_Origin == 'HIM')|
#              (data_snapshot.ProdCd_Origin == 'HEL')|(data_snapshot.ProdCd_Origin == 'PM2')|(data_snapshot.ProdCd_Origin == 'R1L')|
#              (data_snapshot.ProdCd_Origin == 'RRE')|(data_snapshot.ProdCd_Origin == 'LLO')|(data_snapshot.ProdCd_Origin == 'RIN')|
#              (data_snapshot.ProdCd_Origin == 'RAW')|(data_snapshot.ProdCd_Origin == 'RJR')|(data_snapshot.ProdCd_Origin == 'RIV'), 
#              'segment_level_0'] = 1
# data_snapshot.loc[(data_snapshot.ProdCd_Origin == 'AUU')|(data_snapshot.ProdCd_Origin == 'AUN')|
#              (data_snapshot.ProdCd_Origin == 'AUC')|(data_snapshot.ProdCd_Origin == 'REC')|(data_snapshot.ProdCd_Origin == 'LVE'), 
#              'segment_level_0'] = 2
# data_snapshot.loc[(data_snapshot.ProdCd_Origin == 'DEP')|(data_snapshot.ProdCd_Origin == 'PLC')|
#              (data_snapshot.ProdCd_Origin == 'TRB')|(data_snapshot.ProdCd_Origin == 'LIF')|(data_snapshot.ProdCd_Origin == 'VSI'), 
#              'segment_level_0'] = 3
# data_snapshot.loc[(data_snapshot.ProdCd_Origin == 'IIL')|(data_snapshot.ProdCd_Origin == 'EQP')|
#              (data_snapshot.ProdCd_Origin == 'NRE'), 
#              'segment_level_0'] = 4
# data_snapshot.loc[(data_snapshot.ProdCd_Origin == 'PLU')|(data_snapshot.ProdCd_Origin == 'UNS'), 
#              'segment_level_0'] = 0

In [0]:
data_snapshot['segment_level_0'] = 5
data_snapshot.loc[(data_snapshot.ProdCd_YrMo == 'HEQ')|(data_snapshot.ProdCd_YrMo == 'HIM')|
             (data_snapshot.ProdCd_YrMo == 'HEL')|(data_snapshot.ProdCd_YrMo == 'PM2')|(data_snapshot.ProdCd_YrMo == 'R1L')|
             (data_snapshot.ProdCd_YrMo == 'RRE')|(data_snapshot.ProdCd_YrMo == 'LLO')|(data_snapshot.ProdCd_YrMo == 'RIN')|
             (data_snapshot.ProdCd_YrMo == 'RAW')|(data_snapshot.ProdCd_YrMo == 'RJR')|(data_snapshot.ProdCd_YrMo == 'RIV'), 
             'segment_level_0'] = 1
data_snapshot.loc[(data_snapshot.ProdCd_YrMo == 'AUU')|(data_snapshot.ProdCd_YrMo == 'AUN')|
             (data_snapshot.ProdCd_YrMo == 'AUC')|(data_snapshot.ProdCd_YrMo == 'REC')|(data_snapshot.ProdCd_YrMo == 'LVE'), 
             'segment_level_0'] = 2
data_snapshot.loc[(data_snapshot.ProdCd_YrMo == 'DEP')|(data_snapshot.ProdCd_YrMo == 'PLC')|
             (data_snapshot.ProdCd_YrMo == 'TRB')|(data_snapshot.ProdCd_YrMo == 'LIF')|(data_snapshot.ProdCd_YrMo == 'VSI'), 
             'segment_level_0'] = 3
data_snapshot.loc[(data_snapshot.ProdCd_YrMo == 'IIL')|(data_snapshot.ProdCd_YrMo == 'EQP')|
             (data_snapshot.ProdCd_YrMo == 'NRE'), 
             'segment_level_0'] = 4
data_snapshot.loc[(data_snapshot.ProdCd_YrMo == 'PLU')|(data_snapshot.ProdCd_YrMo == 'UNS'), 
             'segment_level_0'] = 0

data_snapshot['segment_level_0'] = data_snapshot['segment_level_0'].astype(int)

#### Create two level segment variables (2): segment_level_1

In [0]:
data_snapshot['segment_level_1'] = 0
# data_snapshot.loc[(data_snapshot['segment_level_0'] == 0)&(data_snapshot['ProdCd_YrMo'] == 'UNS'), 
#                   'segment_level_1'] = 1
# data_snapshot.loc[(data_snapshot['segment_level_0'] == 0)&(data_snapshot['ProdCd_YrMo'] == 'PLU'), 
#                   'segment_level_1'] = 0

data_snapshot.loc[(data_snapshot['ProdCd_YrMo'] == 'UNS'), 'segment_level_1'] = 1
data_snapshot.loc[(data_snapshot['ProdCd_YrMo'] == 'PLU'), 'segment_level_1'] = 0

data_snapshot.loc[(data_snapshot['segment_level_0'] == 3), 'segment_level_1'] = 1
data_snapshot.loc[(data_snapshot['segment_level_0'] == 3 )&(data_snapshot['Term_Origin'] <= 15 ), 
                  'segment_level_1'] = 0
data_snapshot.loc[(data_snapshot['segment_level_0'] == 3) &(data_snapshot['Term_Origin'] > 30),
                  'segment_level_1'] = 2
# data_snapshot['segment_level_1'] = data_snapshot['segment_level_1'].astype(int)

#### Create segment based on secured or not

In [0]:
data_snapshot['segment_level_s'] = 'SECURED'
data_snapshot.loc[(data_snapshot['ProdCd_YrMo'] == 'UNS'), 'segment_level_s'] = 'UNS'
data_snapshot.loc[(data_snapshot['ProdCd_YrMo'] == 'PLU'), 'segment_level_s'] = 'PLU'

In [0]:
data_temp = data_snapshot.loc[(data_snapshot['segment_level_s'] == 'UNS'),]
print(data_temp.segment_level_1.unique())
print(data_temp[['INSTID', 'ProdCd_YrMo', 'segment_level_0', 'segment_level_1', 'Term_Origin']])

[1]
          INSTID ProdCd_YrMo  segment_level_0  segment_level_1  Term_Origin
404    101201387         UNS                0                1        180.0
488    101500217         UNS                0                1         36.0
660    102000280         UNS                0                1         60.0
733    102201427         UNS                0                1         36.0
941    103502459         UNS                0                1         36.0
1130   104401232         UNS                0                1        180.0
1270   105103537         UNS                0                1         61.0
1698   107502672         UNS                0                1        180.0
1786   107902450         UNS                0                1         36.0
2752   112215152         UNS                0                1        180.0
2761   112216288         UNS                0                1         36.0
2916   112401438         UNS                0                1        180.0
3109   1

In [0]:
data_tmp = data_snapshot.loc[data_snapshot['INSTID'] == 100000158, ['ProdCd_YrMo', 'segment_level_0', 'segment_level_1']]
#data_tmp.segment_level_1.unique()
print(data_snapshot.loc[data_snapshot['INSTID'] == 100000158, ['ProdCd_YrMo', 'ProdCd_Orig', 'segment_level_0', 'segment_level_1']])
#print(data_snapshot.segment_level_0.unique())

Empty DataFrame
Columns: [ProdCd_YrMo, ProdCd_Orig, segment_level_0, segment_level_1]
Index: []


#### create utilization ratio for revolving loans

In [0]:
data_snapshot['utilization_rate']=data_snapshot.eval("UNPAID_BALANCE_AMT/(UNPAID_BALANCE_AMT + OFF_BALANCE_AMT)") *100
data_snapshot.loc[(data_snapshot.REVOLVING_FLG == 'N')|(data_snapshot.OFF_BALANCE_AMT + data_snapshot.UNPAID_BALANCE_AMT <= 0), 'utilization_rate' ] = 1


#### Create Interest Rate Indicator

In [0]:
data_snapshot['IntRt_flag_GE15'] = 0
data_snapshot.loc[data_snapshot.EFFECTIVE_INTEREST_RT >= 15, 
                          "IntRt_flag_GE15"] = 1

#### Create age , vintage and date columns

In [0]:
#print(data_snapshot['EOMOpenDt_YrMo'].head())
data_snapshot['OpenDt_Dt'] = pd.to_datetime(data_snapshot['EOMOpenDt_YrMo'].astype(str), format='%d%b%Y')
# data_snapshot['OpenDt_Dt'] = data_snapshot['OpenDt_Dt'] + MonthBegin(1)
data_snapshot['OpenDt_YrMo'] = data_snapshot['OpenDt_Dt'].dt.to_period("M")
data_snapshot["Age_mon"] = data_snapshot['Date_mon'] - data_snapshot['OpenDt_YrMo']
#print(data_snapshot.Age_mon.head())
# data_snapshot["Age_mon"] = data_snapshot["Age_mon"].astype(int)
data_snapshot["Vintage_Days"] = data_snapshot['Date_dt'] - data_snapshot['OpenDt_Dt']
data_snapshot['Vintage_Days2day'] = data_snapshot['Vintage_Days'].astype('timedelta64[D]') + 1
print(data_snapshot['Age_mon'].head())
#data_snapshot["vintage_monthToDate"] = data_snapshot["vintage_monthToDate"].astype(int)

2     95
4     95
6     95
7     94
9    179
Name: Age_mon, dtype: object


In [0]:
# print(data_snapshot.loc[data_snapshot['EOMOpenDt_YrMo'] == , ])
# print(data_snapshot.loc[data_snapshot['OpenDt_Dt'].isnull(), ])
data_snapshot['OpenDt_str'] = data_snapshot['OpenDt_YrMo'].astype(str).str.replace(r'-','')
print(data_snapshot.loc[data_snapshot.Age_mon == 0,['INSTID','Date_mon','EOMOpenDt_YrMo','OpenDt_YrMo','OpenDt_str','MaturityDate_dt']])

          INSTID Date_mon EOMOpenDt_YrMo OpenDt_YrMo OpenDt_str  \
51176  176200449  2019-09      30SEP2019     2019-09     201909   
51177  176200503  2019-09      30SEP2019     2019-09     201909   
51178  176200504  2019-09      30SEP2019     2019-09     201909   
51182  176200534  2019-09      30SEP2019     2019-09     201909   
51184  176200555  2019-09      30SEP2019     2019-09     201909   
51188  176200579  2019-09      30SEP2019     2019-09     201909   
51210  176300487  2019-09      30SEP2019     2019-09     201909   
51213  176300533  2019-09      30SEP2019     2019-09     201909   
51216  176300548  2019-09      30SEP2019     2019-09     201909   
51234  176400438  2019-09      30SEP2019     2019-09     201909   
51240  176400502  2019-09      30SEP2019     2019-09     201909   
51241  176400517  2019-09      30SEP2019     2019-09     201909   
51262  176500454  2019-09      30SEP2019     2019-09     201909   
51292  176600470  2019-09      30SEP2019     2019-09     20190

In [0]:
# print(data_snapshot.loc[data_snapshot.Age_mon == 0,['INSTID','Date_mon','EOMOpenDt_YrMo','OpenDt_YrMo']])

In [0]:
data_snapshot_1 = data_snapshot.loc[data_snapshot['MaturityDate_dt'] > '2019-09-30',]
print(data_snapshot_1.shape[0], data_snapshot.shape[0])

27356 27950


In [0]:
data_snapshot_1.loc[data_snapshot_1.INSTID == 1113180, 'EOMMaturityDt_YrMo']

Series([], Name: EOMMaturityDt_YrMo, dtype: object)

In [0]:
data_forecast = data_snapshot_1[  
    ['INSTID',
    'ProdCd_YrMo',
    'EOMOpenDt_YrMo',
    'OpenDt_str',
    'EOMMaturityDt_YrMo',
    'MaturityDate_mon',
    'Term_Origin',
    'EFFECTIVE_INTEREST_RT',
    'RevolveIn',
    'segment_level_0',
    'segment_level_1',
    'segment_level_s',
    'vintage_year',  
    'IntRt_flag_GE15',
    'utilization_rate',
    'PMT_L6MO',
    'Age_mon',
    'OpenDt_Dt',
    'OpenDt_YrMo',
    'Vintage_Days',
    'MaturityDate_dt',
    'NetGLOutstandingBalance',
    'OpenAmt_YrMo',
    'PrinPayment_L6mon',
    'UNPAID_BALANCE_AMT', 
    'OFF_BALANCE_AMT',
    'DAYS_PAST_DUE_CNT',
    'RiskCd',
    'EOMChrgOffDt'
]]
#'EOMOpenDt_YrMo_dt','EOMOpenDt_YrMo_mon','EOMMaturityDt_YrMo_dt','EOMMaturityDt_YrMo_mon','PA_segment','ProdCd_flag_PLC','ProdCd_flag_PLU','ProdCd_flag_UNS','GrossOutstandingBalance',
# 'MaturityDate_dt','Date_dt','Date_mon',

### expand the forecast series for each loan point in the forecast sample

In [0]:
print(data_forecast.loc[data_forecast['INSTID'] == 620202,])

Empty DataFrame
Columns: [INSTID, ProdCd_YrMo, EOMOpenDt_YrMo, OpenDt_str, EOMMaturityDt_YrMo, MaturityDate_mon, OpenAmt_YrMo, Term_Origin, EFFECTIVE_INTEREST_RT, RevolveIn, segment_level_0, segment_level_1, vintage_year, IntRt_flag_GE15, utilization_rate, PMT_L6MO, Age_mon, OpenDt_Dt, OpenDt_YrMo, Vintage_Days, MaturityDate_dt]
Index: []

[0 rows x 21 columns]


In [0]:

# acctid_list = data_forecast.loc[data_forecast['MaturityDate_dt'].notnull()]['INSTID']
#print(acctid_list)

acctid_list = data_forecast['INSTID']

#### we have one loan in the portfolio 2019Jun30 snapshot that has MaturityDate_dt missing. So, in general, for loans that have maturity date information missing from the portfolio data, for this particular loan the forecast period is automatically set to the maximum number of forecast period which is 360 periods. 

In [0]:
%%time
var_cols = list(data_forecast.columns)
df_forecast = pd.DataFrame()
for id in acctid_list:
    data_sub = data_forecast.loc[data_forecast.INSTID == id, ]
    if pd.isnull(data_sub.iloc[0]['MaturityDate_dt']):
        forecast_period = 360
    else:
        Maturity_date = data_sub.iloc[0]['MaturityDate_dt']
        forecast_period = Maturity_date.to_period('M') - pd.to_datetime('2019-09-30').to_period('M')
    #print(Maturity_date)
    AcctID = data_sub.iloc[0]['INSTID']
    forecast_period_df = pd.DataFrame(index = pd.date_range('2019-10-31', periods = forecast_period, freq = 'M')).reset_index()
    forecast_period_df['AcctID'] = data_sub.iloc[0]['INSTID']
    df_forecast = df_forecast.append(forecast_period_df, ignore_index = True)
    

Wall time: 7min 12s


In [0]:
print(df_forecast.loc[df_forecast.AcctID == 620202,])

Empty DataFrame
Columns: [index, AcctID]
Index: []


#### merge the snapshot data with the forecast dataset

In [0]:
# df_forecast_0 = df_forecast.groupby('AcctID').head(30)
# df_forecast_1 = pd.merge(df_forecast_0, data_forecast, left_on ='AcctID', right_on = 'INSTID',  how = 'left')

df_forecast_1 = pd.merge(df_forecast, data_forecast, left_on ='AcctID', right_on = 'INSTID',  how = 'left')

In [0]:
print(df_forecast_1)

            index     AcctID     INSTID ProdCd_YrMo EOMOpenDt_YrMo OpenDt_str  \
0      2019-10-31  100000170  100000170         PLU      31OCT2011     201110   
1      2019-11-30  100000170  100000170         PLU      31OCT2011     201110   
2      2019-12-31  100000170  100000170         PLU      31OCT2011     201110   
3      2020-01-31  100000170  100000170         PLU      31OCT2011     201110   
4      2020-02-29  100000170  100000170         PLU      31OCT2011     201110   
5      2020-03-31  100000170  100000170         PLU      31OCT2011     201110   
6      2020-04-30  100000170  100000170         PLU      31OCT2011     201110   
7      2020-05-31  100000170  100000170         PLU      31OCT2011     201110   
8      2020-06-30  100000170  100000170         PLU      31OCT2011     201110   
9      2020-07-31  100000170  100000170         PLU      31OCT2011     201110   
10     2020-08-31  100000170  100000170         PLU      31OCT2011     201110   
11     2020-09-30  100000170

In [0]:
# recalculate the age for the forecast dataset
df_forecast_1.reset_index()
df_forecast_1['forecast_YrMo'] = df_forecast_1['index'].dt.to_period("M")
df_forecast_1['forecast_mon'] = df_forecast_1.forecast_YrMo - df_forecast_1.OpenDt_YrMo
df_forecast_1['ssn_mon'] = df_forecast_1.forecast_YrMo.dt.strftime("%m").astype(int)
print(list(df_forecast_1.ssn_mon.head()))


[10, 11, 12, 1, 2]


In [0]:
# print(df_forecast_1.forecast_YrMo)

0        2019-07
1        2019-08
2        2019-09
3        2019-10
4        2019-11
5        2019-07
6        2019-08
7        2019-09
8        2019-10
9        2019-11
10       2019-12
11       2020-01
12       2020-02
13       2020-03
14       2020-04
15       2019-07
16       2019-08
17       2019-09
18       2019-10
19       2019-11
20       2019-07
21       2019-08
22       2019-09
23       2019-10
24       2019-11
25       2019-12
26       2020-01
27       2019-07
28       2019-08
29       2019-09
           ...  
426351   2021-02
426352   2021-03
426353   2021-04
426354   2021-05
426355   2021-06
426356   2019-07
426357   2019-08
426358   2019-09
426359   2019-10
426360   2019-11
426361   2019-12
426362   2020-01
426363   2020-02
426364   2020-03
426365   2020-04
426366   2020-05
426367   2020-06
426368   2020-07
426369   2020-08
426370   2020-09
426371   2020-10
426372   2020-11
426373   2020-12
426374   2021-01
426375   2019-07
426376   2019-08
426377   2019-09
426378   2019-

In [0]:
print(df_forecast_1.loc[df_forecast_1['AcctID'] == 100000170,])

         index     AcctID     INSTID ProdCd_YrMo EOMOpenDt_YrMo OpenDt_str  \
154 2019-10-31  100000170  100000170         PLU      31OCT2011     201110   
155 2019-11-30  100000170  100000170         PLU      31OCT2011     201110   
156 2019-12-31  100000170  100000170         PLU      31OCT2011     201110   
157 2020-01-31  100000170  100000170         PLU      31OCT2011     201110   
158 2020-02-29  100000170  100000170         PLU      31OCT2011     201110   
159 2020-03-31  100000170  100000170         PLU      31OCT2011     201110   
160 2020-04-30  100000170  100000170         PLU      31OCT2011     201110   
161 2020-05-31  100000170  100000170         PLU      31OCT2011     201110   
162 2020-06-30  100000170  100000170         PLU      31OCT2011     201110   
163 2020-07-31  100000170  100000170         PLU      31OCT2011     201110   
164 2020-08-31  100000170  100000170         PLU      31OCT2011     201110   
165 2020-09-30  100000170  100000170         PLU      31OCT2011 

In [0]:
# df_raw_macro = pd.read_excel('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\Basket_2019-09-12_3h_11m2.csv', 
#                              sheetname='working', header=4,  parse_dates=True)

# df_raw_macro = pd.read_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\macro_Naz_2019-09-20.csv')

df_raw_macro = pd.read_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\Basket_2019-09-23.csv')


# print(df_raw_macro)
#index_col=0,

In [0]:
# df_raw_macro['gspTX_22m_LogRatio_3m_lag'] = np.log(df_raw_macro['gspTX'].shift(3)/df_raw_macro['gspTX'].shift(22+3))
# df_raw_macro['gspTX_22m_LogRatio_3m_lag'] = df_raw_macro['gspTX_22m_LogRatio_3m_lag'].shift(3)

In [0]:
# df_raw_macro['gspTX_22m_LogRatio_3m_lag'] = np.log(df_raw_macro['gspTX'].shift(3)/df_raw_macro['gspTX'].shift(22+3))
df_raw_macro['gspTX_22m_LogRatio_3m_lag'] = np.log((df_raw_macro['gspTX'])/df_raw_macro['gspTX'].shift(22))
df_raw_macro['gspTX_22m_LogRatio_3m_lag'] = df_raw_macro['gspTX_22m_LogRatio_3m_lag'].shift(3)
# log(waterfall(_horizon_-3,_FGDPQ_.state1,FGDP)/waterfall(_horizon_-3-22,_FGDPQ_.state1,FGDP))

df_raw_macro['gspTX_12m_LogRatio_7m_change_11m_lag'] = np.log(df_raw_macro['gspTX']/df_raw_macro['gspTX'].shift(12))
df_raw_macro['gspTX_12m_LogRatio_7m_change_11m_lag'] = df_raw_macro['gspTX_12m_LogRatio_7m_change_11m_lag'] - \
                                                    df_raw_macro['gspTX_12m_LogRatio_7m_change_11m_lag'].shift(7)
df_raw_macro['gspTX_12m_LogRatio_7m_change_11m_lag'] = df_raw_macro['gspTX_12m_LogRatio_7m_change_11m_lag'].shift(11)

df_raw_macro['itbyBAA_24m_Diff_12m_lag'] = df_raw_macro['itbyBAA'] - df_raw_macro['itbyBAA'].shift(12)
df_raw_macro['itbyBAA_24m_Diff_12m_lag'] = df_raw_macro['itbyBAA_24m_Diff_12m_lag'].shift(24)

df_raw_macro['byAAA_9m_lag'] = df_raw_macro['byAAA'].shift(9)


In [0]:
print(df_raw_macro.loc[df_raw_macro['itbyBAA_24m_Diff_12m_lag'].notnull(),])

           DATE      gspTX  itbyBAA   byAAA  gspTX_22m_LogRatio_3m_lag  \
90   07/31/1997   614.9826   6.9886  7.3181                   0.150492   
91   08/31/1997   618.3656   6.8491  7.1476                   0.153037   
92   09/30/1997   621.0180   6.8010  7.0311                   0.154157   
93   10/31/1997   623.2701   6.8001  6.9525                   0.154302   
94   11/30/1997   625.4666   6.7758  6.8799                   0.153718   
95   12/31/1997   627.8512   6.6843  6.7933                   0.152532   
96   01/31/1998   630.4040   6.5627  6.7070                   0.151205   
97   02/28/1998   632.8047   6.4814  6.6523                   0.149212   
98   03/31/1998   635.0946   6.4735  6.6396                   0.146020   
99   04/30/1998   637.4799   6.5082  6.6492                   0.140881   
100  05/31/1998   640.0912   6.5298  6.6493                   0.134564   
101  06/30/1998   643.0719   6.4954  6.6169                   0.129333   
102  07/31/1998   646.3777   6.4079  6

In [0]:
df_raw_macro.reset_index()
#df_raw_macro['forecast_YrMo'] = df_raw_macro['Date'].dt.to_period('M')
df_raw_macro['forecast_YrMo'] = pd.to_datetime(df_raw_macro['DATE'].astype(str), format='%m/%d/%Y')
df_raw_macro['forecast_YrMo'] = df_raw_macro['forecast_YrMo'].dt.to_period('M')
# df_transformed_macro = df_raw_macro.loc[df_raw_macro['itbyBAA_24m_Diff_12m_lag'].notnull(),]
# df_itby = df_transformed_macro[['forecast_YrMo','itbyBAA_24m_Diff_12m_lag']]
# print(df_itby)
# print(df_forecast_11['forecast_YrMo'].unique())
print(df_raw_macro['forecast_YrMo'].unique())
# print(df_transformed_macro['forecast_YrMo'].dtype)

[Period('1990-01', 'M') Period('1990-02', 'M') Period('1990-03', 'M')
 Period('1990-04', 'M') Period('1990-05', 'M') Period('1990-06', 'M')
 Period('1990-07', 'M') Period('1990-08', 'M') Period('1990-09', 'M')
 Period('1990-10', 'M') Period('1990-11', 'M') Period('1990-12', 'M')
 Period('1991-01', 'M') Period('1991-02', 'M') Period('1991-03', 'M')
 Period('1991-04', 'M') Period('1991-05', 'M') Period('1991-06', 'M')
 Period('1991-07', 'M') Period('1991-08', 'M') Period('1991-09', 'M')
 Period('1991-10', 'M') Period('1991-11', 'M') Period('1991-12', 'M')
 Period('1992-01', 'M') Period('1992-02', 'M') Period('1992-03', 'M')
 Period('1992-04', 'M') Period('1992-05', 'M') Period('1992-06', 'M')
 Period('1992-07', 'M') Period('1992-08', 'M') Period('1992-09', 'M')
 Period('1992-10', 'M') Period('1992-11', 'M') Period('1992-12', 'M')
 Period('1993-01', 'M') Period('1993-02', 'M') Period('1993-03', 'M')
 Period('1993-04', 'M') Period('1993-05', 'M') Period('1993-06', 'M')
 Period('1993-07', '

In [0]:
# df_forecast_2 = pd.merge(df_forecast_11, df_itby, on ='forecast_YrMo', how = 'left')
df_forecast_2 = pd.merge(df_forecast_1, df_raw_macro, on = 'forecast_YrMo', how = 'left')
# df_forecast_20 = pd.merge(df_forecast_2, df_itby, on = 'forecast_YrMo', how = 'left')
# df_forecast_2['Vintage_YrMo'] = df_forecast_2['OpenDt_str'].astype(int)


# df_forecast_2['Vintage_year'] = df_forecast_2['OpenDt_YrMo'].dt.strftime("%Y")
# df_forecast_2['Vintage_mon'] = df_forecast_2['OpenDt_YrMo'].dt.strftime("%m")
# df_forecast_2['Vintage_YrMo'] = df_forecast_2['Vintage_year'].astype(int) * 100 + df_forecast_2['Vintage_mon'].astype(int)

In [0]:
# print(df_forecast_11['forecast_YrMo'].dtype)
# print(df_forecast_11.loc[df_forecast_11['AcctID'] == 620202,])
# print(df_raw_macro.loc[df_raw_macro['DATE'] >= '07/31/2019',])
print(df_forecast_2.columns)
# print(df_forecast_2.loc[df_forecast_2['itbyBAA_24m_Diff_12m_lag'].notnull(),].forecast_YrMo.unique())

Index(['index', 'AcctID', 'INSTID', 'ProdCd_YrMo', 'EOMOpenDt_YrMo',
       'OpenDt_str', 'EOMMaturityDt_YrMo', 'MaturityDate_mon', 'Term_Origin',
       'EFFECTIVE_INTEREST_RT', 'RevolveIn', 'segment_level_0',
       'segment_level_1', 'segment_level_s', 'vintage_year', 'IntRt_flag_GE15',
       'utilization_rate', 'PMT_L6MO', 'Age_mon', 'OpenDt_Dt', 'OpenDt_YrMo',
       'Vintage_Days', 'MaturityDate_dt', 'NetGLOutstandingBalance',
       'OpenAmt_YrMo', 'PrinPayment_L6mon', 'UNPAID_BALANCE_AMT',
       'OFF_BALANCE_AMT', 'DAYS_PAST_DUE_CNT', 'RiskCd', 'EOMChrgOffDt',
       'forecast_YrMo', 'forecast_mon', 'ssn_mon', 'DATE', 'gspTX', 'itbyBAA',
       'byAAA', 'gspTX_22m_LogRatio_3m_lag',
       'gspTX_12m_LogRatio_7m_change_11m_lag', 'itbyBAA_24m_Diff_12m_lag',
       'byAAA_9m_lag'],
      dtype='object')


In [0]:
# print(df_forecast_2[['Vintage_year', 'Vintage_mon', 'Vintage_YrMo', 'OpenDt_str' ]])

In [0]:
# df_forecast_2.loc[df_forecast_2.OpenDt_YrMo.isnull(),] 

In [0]:
# print(df_forecast_2)

#### load the PA and PD lookup tables.
##### PD and PA lookup tables need to be updated especially: PD/PA vintage tables (to incorporate some very old loans that were excluded in the model development stage), PA Time table (to incorporate more forecasting periods), 
##### All the PD lookup tables are generated using the file "PD_in_out_sample_test_201901-final
##### All the PA lookup tables are generated using the file 'PA_in_out_sample_test_201901-final

In [0]:
#loc = ("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP")
df_PA_age = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190913.xlsx",
                           sheetname='PA_age')
df_PA_ssn = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190913.xlsx",
                           sheetname='PA_ssn')
df_PA_vintage = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190913.xlsx",
                           sheetname='PA_vintage')
df_PA_time = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190913.xlsx",
                           sheetname='PA_time')

In [0]:
df_PD_age = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190913.xlsx",
                           sheetname='PD_age')
df_PD_ssn = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190913.xlsx",
                           sheetname='PD_ssn')
df_PD_vintage = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190913.xlsx",
                           sheetname='PD_vintage')


In [0]:
#print(df_PA_age)

In [0]:
### This section of the code is to check the old vintages in the snapshot data that is out of the range for the age lookup
### tables. 
### The current strategy is to fill the "pa_age" and "pd_age" tables for the older vintages that were previously not in the 
### lookup tables
data_vintage = data_snapshot
data_vintage['vintage_YrMo']  = (pd.to_datetime(data_vintage['EOMOpenDt_YrMo'].astype(str), format='%d%b%Y')).dt.to_period('M')
data_vintage['vintage_mon'] = data_vintage['vintage_YrMo'].dt.strftime('%m')
old_vintage = data_vintage.loc[data_vintage.vintage_year < '1995',]
old_vintage['pd_vint_lookup_key'] = old_vintage['vintage_year'].astype(int)*100 + old_vintage['vintage_mon'].astype(int)
pd_old_vintage = old_vintage[['pd_vint_lookup_key']].sort_values(by=['pd_vint_lookup_key'])
pd_old_vintage = pd_old_vintage.drop_duplicates()

old_vintage['vint_seg_key'] = old_vintage['pd_vint_lookup_key'].astype(str) + '-' + old_vintage['segment_level_0'].astype(str) + '-' + old_vintage['segment_level_1'].astype(str)
# print(list(old_vintage.columns))
# print(old_vintage[['INSTID', 'vintage_YrMo','EOMOpenDt_YrMo','vintage_mon','vintage_year', 'EOMOpenDt_YrMo','pd_vint_lookup_key', 'vint_seg_key']])
print(pd_old_vintage)



       pd_vint_lookup_key
53273              196405
55014              196902
60564              197209
9987               197407
63088              197409
2089               197508
53354              197601
56184              197604
57097              197610
62043              197612
54442              197705
59446              197803
59442              197806
55830              197904
4845               197906
54457              197908
55313              198009
9817               198011
54532              198106
55740              198107
45684              198109
2330               198110
578                198111
40400              198204
60619              198205
45692              198206
54866              198207
54683              198208
32537              198209
2113               198210
...                   ...
57305              199207
10891              199208
2294               199209
10962              199210
57648              199211
102                199212
54839       

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [0]:
### Below is to populate the PD_Age table for those old vintages by taking the the age component from the "1995-01-01"
pd_old_vintage['pd_vintage_component'] = df_PD_vintage.iloc[0]['pd_vintage_component']
print(pd_old_vintage)
df_PD_vint_ext = pd_old_vintage.append(df_PD_vintage, ignore_index = True)
df_PD_vint_ext = df_PD_vint_ext.sort_values(by = ['pd_vint_lookup_key'])
print(df_PD_vint_ext)

       pd_vint_lookup_key  pd_vintage_component
53273              196405              0.870665
55014              196902              0.870665
60564              197209              0.870665
9987               197407              0.870665
63088              197409              0.870665
2089               197508              0.870665
53354              197601              0.870665
56184              197604              0.870665
57097              197610              0.870665
62043              197612              0.870665
54442              197705              0.870665
59446              197803              0.870665
59442              197806              0.870665
55830              197904              0.870665
4845               197906              0.870665
54457              197908              0.870665
55313              198009              0.870665
9817               198011              0.870665
54532              198106              0.870665
55740              198107              0

In [0]:
old_vint_pa = old_vintage[['vint_seg_key', 'segment_level_0', 'segment_level_1', 'vintage_year', 'vintage_mon']]
old_vint_pa['_name_1'] =str(199501) + old_vint_pa['vint_seg_key'].str.slice(6,10) 
old_vint_list = list(old_vint_pa['_name_1'].unique())

df_PA_vint_1995 = df_PA_vintage[df_PA_vintage['_name_'].isin(old_vint_list)]
old_vint_pa = pd.merge(old_vint_pa, df_PA_vint_1995, left_on = '_name_1', right_on ='_name_',how = 'left')

old_vint_pa_1 = old_vint_pa[['_name_', 'pa_vint_component', 'vint_seg_key']]
old_vint_pa_1['_name_'] = old_vint_pa_1['vint_seg_key']
old_vint_pa_1 = old_vint_pa_1.drop_duplicates().sort_values(by = ['_name_']).drop(['vint_seg_key'], axis = 1)
# print(old_vint_pa_1)


df_PA_vint_ext = old_vint_pa_1.append(df_PA_vintage, ignore_index = True)

# vint_seg_key_1995 = [str(199501) + s for s in old_vint_list]

# os.chdir('C:\\Users\\Hu09559\\Documents\\Consumer\\Output\\Models\\PD')
# writer = pd.ExcelWriter('PDPA_Tables_SAS_201909.xlsx')
# df_PA_vint_ext.to_excel(writer, sheet_name='PA_vintage')
# df_PD_vint_ext.to_excel(writer, sheet_name='PD_vintage')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
# print(df_PA_vint_ext)

#### we need to merge the lookup tables with the df_forecast_2 dataset

In [0]:
df_PD_vint_ext['pd_vint_lookup_key'] = df_PD_vint_ext['pd_vint_lookup_key'].astype(str)
df_forecast_21 = pd.merge(df_forecast_2, df_PD_age, left_on = 'forecast_mon', right_on = 'pd_age_lookup_key', how = 'left')
df_forecast_22 = pd.merge(df_forecast_21, df_PD_ssn, left_on = 'ssn_mon', right_on = 'pd_ssn_lookup_key', how = 'left')
df_forecast_23 = pd.merge(df_forecast_22, df_PD_vint_ext, left_on = 'OpenDt_str', right_on = 'pd_vint_lookup_key', how = 'left')


In [0]:
# print(df_forecast_22.pd_ssn_component)

In [0]:

df_forecast_23['age_seg_key'] = df_forecast_23['forecast_mon'].astype(str) + '-' + df_forecast_23['segment_level_0'].astype(str) + '-' + df_forecast_23['segment_level_1'].astype(str)
df_forecast_23['vint_seg_key'] = df_forecast_23['OpenDt_str'] + '-' + df_forecast_23['segment_level_0'].astype(str) + '-' + df_forecast_23['segment_level_1'].astype(str)
df_forecast_23['ssn_seg_key'] = df_forecast_23['ssn_mon'].astype(str) + '-' + df_forecast_23['segment_level_0'].astype(str) + '-' + df_forecast_23['segment_level_1'].astype(str)

df_forecast_23['forecast_YrMo_1'] = df_forecast_23['forecast_YrMo'].dt.strftime("%Y").astype(int) * 100 +df_forecast_23['forecast_YrMo'].dt.strftime("%m").astype(int)
df_forecast_23['time_seg_key'] = df_forecast_23['forecast_YrMo_1'].astype(str) + '-' + df_forecast_23['segment_level_0'].astype(str) + '-' + df_forecast_23['segment_level_1'].astype(str)




In [0]:
print(df_forecast_27.loc[df_forecast_23.AcctID == 10000128 ,['forecast_YrMo','forecast_YrMo_1','age_seg_key','pa_age_component'] ])

   forecast_YrMo  forecast_YrMo_1 age_seg_key  pa_age_component
59       2019-10           201910     180-0-0         -5.547172
60       2019-11           201911     181-0-0         -5.547172
61       2019-12           201912     182-0-0         -5.547172
62       2020-01           202001     183-0-0         -5.547172
63       2020-02           202002     184-0-0         -5.547172
64       2020-03           202003     185-0-0         -5.547172
65       2020-04           202004     186-0-0         -5.547172
66       2020-05           202005     187-0-0         -5.547172
67       2020-06           202006     188-0-0         -5.547172
68       2020-07           202007     189-0-0         -5.547172
69       2020-08           202008     190-0-0         -5.547172
70       2020-09           202009     191-0-0         -5.547172
71       2020-10           202010     192-0-0         -5.547172


In [0]:
df_forecast_24 = pd.merge(df_forecast_23, df_PA_age, left_on = 'age_seg_key', right_on = '_name_', how = 'left')
df_forecast_25 = pd.merge(df_forecast_24, df_PA_ssn, left_on = 'ssn_seg_key', right_on = '_name_', how = 'left')
df_forecast_26 = pd.merge(df_forecast_25, df_PA_vint_ext, left_on = 'vint_seg_key', right_on = '_name_', how = 'left')
df_forecast_27 = pd.merge(df_forecast_26, df_PA_time, left_on = 'time_seg_key', right_on = '_name_', how = 'left')
# test = df_forecast_27.loc[df_forecast_27['AcctID'] == 100000170, ['AcctID','pa_time_component','vint_seg_key','Vintage_YrMo','pa_vintage_component','pa_ssn_component','pa_age_component']]

In [0]:
print(df_PA_age.loc[df_PA_age['_name_'] == '177-0-0',])

     pa_age_component   _name_
176         -5.547172  177-0-0


#### calculation for PD and PA

In [0]:
df_forecast_27['PA'] = np.exp(df_forecast_27['pa_age_component'] + df_forecast_27['pa_vint_component'] + df_forecast_27['pa_time_component'] + df_forecast_27['pa_ssn_component'])/(1 + np.exp(df_forecast_27['pa_age_component'] + df_forecast_27['pa_vint_component'] + df_forecast_27['pa_time_component'] + df_forecast_27['pa_ssn_component']))

In [0]:
df_forecast_27['pd_time_component'] = 0.159128755 + (-2.571780307) * df_forecast_27['gspTX_22m_LogRatio_3m_lag'] + 0.047804409\
* df_forecast_27['itbyBAA_24m_Diff_12m_lag']
df_forecast_27['PD'] = np.exp(df_forecast_27['pd_age_component'] + df_forecast_27['pd_vintage_component'] + df_forecast_27['pd_time_component'] + df_forecast_27['pd_ssn_component'])/\
(1 + np.exp(df_forecast_27['pd_age_component'] + df_forecast_27['pd_vintage_component'] + df_forecast_27['pd_time_component'] + df_forecast_27['pd_ssn_component']))

In [0]:
# print(df_forecast_27.loc[df_forecast_27['itbyBAA_24m_Diff_12m_lag'].notnull(),])

In [0]:
# df_forecast_Naz = df_forecast_27[['AcctID', 'OpenDt_yrMo', 'RevovleIn', 'segment_level_0', '']]
# df_forecast_27.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\CompareResults_withNaz20190918.csv')

#### upload the the ccf file

In [0]:
df_forecast_27['ProdCd_recovery'] = df_forecast_27['ProdCd_YrMo']
df_forecast_27.loc[~df_forecast_27['ProdCd_YrMo'].isin(['AUC','AUN','AUU','DEP','PLC','PLU','REC','TRB','UNS']), 'ProdCd_recovery'] = 'Other'
print(df_forecast_27.ProdCd_recovery.unique())
df_recovery = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190521.xlsx",
                           sheetname='LGD_recovery_table')
print(df_recovery)
df_forecast_271 = pd.merge(df_forecast_27, df_recovery, left_on = 'ProdCd_recovery', right_on = 'ProdCd_YrMo', how = 'left')
# print(df_ccf)

# df_ccf = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190521.xlsx",
#                            sheetname='Utilization_table')
# df_ccf.rename(columns = {'Utilization_Rate':'ccf'}, inplace = True)
# print(df_ccf)

# df_forecast_28 = pd.merge(df_forecast_271, df_ccf,  left_on = 'forecast_mon', right_on = 'Age_mon', how = 'left')
df_forecast_28 = df_forecast_271

['PLU' 'PLC' 'DEP' 'UNS' 'TRB' 'REC' 'AUU' 'AUN' 'Other' 'AUC']
  ProdCd_YrMo  Recovery
0         AUC  0.175128
1         AUN  0.329974
2         AUU  0.183022
3         DEP  0.209876
4       Other  0.148290
5         PLC  0.106800
6         PLU  0.263097
7         REC  0.173711
8         TRB  0.853274
9         UNS  0.357557


In [0]:
# print(df_forecast_28.loc[df_forecast_28['ProdCd_recovery'] == 'Other',])
print(df_forecast_28.columns)

Index(['index', 'AcctID', 'INSTID', 'ProdCd_YrMo_x', 'EOMOpenDt_YrMo',
       'OpenDt_str', 'EOMMaturityDt_YrMo', 'MaturityDate_mon', 'Term_Origin',
       'EFFECTIVE_INTEREST_RT', 'RevolveIn', 'segment_level_0',
       'segment_level_1', 'segment_level_s', 'vintage_year', 'IntRt_flag_GE15',
       'utilization_rate', 'PMT_L6MO', 'Age_mon', 'OpenDt_Dt', 'OpenDt_YrMo',
       'Vintage_Days', 'MaturityDate_dt', 'NetGLOutstandingBalance',
       'OpenAmt_YrMo', 'PrinPayment_L6mon', 'UNPAID_BALANCE_AMT',
       'OFF_BALANCE_AMT', 'DAYS_PAST_DUE_CNT', 'RiskCd', 'EOMChrgOffDt',
       'forecast_YrMo', 'forecast_mon', 'ssn_mon', 'DATE', 'gspTX', 'itbyBAA',
       'byAAA', 'gspTX_22m_LogRatio_3m_lag',
       'gspTX_12m_LogRatio_7m_change_11m_lag', 'itbyBAA_24m_Diff_12m_lag',
       'byAAA_9m_lag', 'pd_age_component', 'pd_age_lookup_key',
       'pd_ssn_lookup_key', 'pd_ssn_component', 'pd_vint_lookup_key',
       'pd_vintage_component', 'age_seg_key', 'vint_seg_key', 'ssn_seg_key',
       'for

#### Define the variables that go into the calculation of LGD 

In [0]:
# df_forecast_28 = pd.merge(df_forecast_27, df_ccf, left_on = 'ProdCd_recovery', right_on = 'ProdCd_YrMo', how = 'left')
print(df_forecast_28['RevolveIn'].dtypes)


df_forecast_28['Intercept'] = 1
df_forecast_28['ProdCd_flag_PLC'] = 0
df_forecast_28.loc[df_forecast_28['ProdCd_YrMo_x'] == 'PLC','ProdCd_flag_PLC'] = 1
df_forecast_28['ProdCd_flag_PLU'] = 0
df_forecast_28.loc[df_forecast_28['ProdCd_YrMo_x'] == 'PLU','ProdCd_flag_PLU'] = 1
df_forecast_28['ProdCd_flag_UNS'] = 0
df_forecast_28.loc[df_forecast_28['ProdCd_YrMo_x'] == 'UNS','ProdCd_flag_UNS'] = 1
# data_snapshot["Age_mon"] = data_snapshot['Date_mon'] - data_snapshot['OpenDt_YrMo']
df_forecast_28['Age'] = df_forecast_28['forecast_YrMo'] - df_forecast_28['OpenDt_YrMo']
df_forecast_28['Revolve_1'] = 1
df_forecast_28.loc[df_forecast_28['RevolveIn'] == 0, 'Revolve_1'] = 0

df_forecast_28['utilization_rate'] = df_forecast_28['UNPAID_BALANCE_AMT']/(df_forecast_28['UNPAID_BALANCE_AMT'] + df_forecast_28['OFF_BALANCE_AMT'])
df_forecast_28.loc[(df_forecast_28['Revolve_1'] == 0) | ((df_forecast_28['UNPAID_BALANCE_AMT'] + df_forecast_28['OFF_BALANCE_AMT']) <= 0 ) , 'utilization_rate'] = 1

# calculation for LGD
df_forecast_28['LGD'] = df_forecast_28['Intercept'] *(-4.0095893) + df_forecast_28['ProdCd_flag_PLC'] * (5.6577647)\
+ df_forecast_28['ProdCd_flag_PLU'] * (6.6497561) + df_forecast_28['ProdCd_flag_UNS'] * (0.9895465) \
+ df_forecast_28['IntRt_flag_GE15'] * (2.6182774) + df_forecast_28['utilization_rate'] * (0.0163606)\
+ df_forecast_28['Age'] * (0.0073012) + df_forecast_28['Revolve_1'] * (0.6268364) \
+ df_forecast_28['byAAA_9m_lag'] * (-10.9540513) + df_forecast_28['gspTX_12m_LogRatio_7m_change_11m_lag'] * (-4.5056069)

df_forecast_28['LGD_CGD'] = 1/(1 + np.exp(-df_forecast_28['LGD'].astype(float)))

print(df_forecast_28.columns)



int64
Index(['index', 'AcctID', 'INSTID', 'ProdCd_YrMo_x', 'EOMOpenDt_YrMo',
       'OpenDt_str', 'EOMMaturityDt_YrMo', 'MaturityDate_mon', 'Term_Origin',
       'EFFECTIVE_INTEREST_RT', 'RevolveIn', 'segment_level_0',
       'segment_level_1', 'segment_level_s', 'vintage_year', 'IntRt_flag_GE15',
       'utilization_rate', 'PMT_L6MO', 'Age_mon', 'OpenDt_Dt', 'OpenDt_YrMo',
       'Vintage_Days', 'MaturityDate_dt', 'NetGLOutstandingBalance',
       'OpenAmt_YrMo', 'PrinPayment_L6mon', 'UNPAID_BALANCE_AMT',
       'OFF_BALANCE_AMT', 'DAYS_PAST_DUE_CNT', 'RiskCd', 'EOMChrgOffDt',
       'forecast_YrMo', 'forecast_mon', 'ssn_mon', 'DATE', 'gspTX', 'itbyBAA',
       'byAAA', 'gspTX_22m_LogRatio_3m_lag',
       'gspTX_12m_LogRatio_7m_change_11m_lag', 'itbyBAA_24m_Diff_12m_lag',
       'byAAA_9m_lag', 'pd_age_component', 'pd_age_lookup_key',
       'pd_ssn_lookup_key', 'pd_ssn_component', 'pd_vint_lookup_key',
       'pd_vintage_component', 'age_seg_key', 'vint_seg_key', 'ssn_seg_key',
     

In [0]:
print(df_forecast_28['LGD_CGD'])

0          1.101544e-20
1          5.295678e-20
2          1.364407e-19
3          2.470574e-19
4          5.289467e-19
5          1.844803e-18
6          5.023062e-17
7          1.257936e-16
8          2.474669e-17
9          1.151290e-18
10         7.511905e-20
11         2.607416e-20
12         4.889174e-20
13         3.034726e-19
14         4.204374e-18
15         8.610921e-17
16         1.633934e-15
17         1.980156e-14
18         1.418024e-13
19         6.332126e-13
20         1.701343e-12
21         2.961377e-12
22         3.398560e-12
23         2.735350e-12
24         1.862120e-12
25         1.383740e-12
26         1.256603e-12
27         1.241825e-12
28         1.022011e-12
29         5.998863e-13
               ...     
3622567    3.076332e-12
3622568    3.728189e-11
3622569    2.669821e-10
3622570    1.192197e-09
3622571    3.203248e-09
3622572    5.575608e-09
3622573    6.398727e-09
3622574    1.753357e-17
3622575    8.429274e-17
3622576    2.171764e-16
3622577    3.932

In [0]:
#df_forecast_28.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\CompareResults_withNaz20190920.csv')

In [0]:
print(df_forecast_28[['AcctID','forecast_YrMo','Intercept','ProdCd_flag_PLC','ProdCd_flag_PLU','ProdCd_flag_UNS','IntRt_flag_GE15','utilization_rate','gspTX_12m_LogRatio_7m_change_11m_lag', 'byAAA_9m_lag','Age','Revolve_1','PD','LGD_CGD'] ])

            AcctID forecast_YrMo  Intercept  ProdCd_flag_PLC  ProdCd_flag_PLU  \
0        100000158       2019-10          1                0                0   
1        100000158       2019-11          1                0                0   
2        100000158       2019-12          1                0                0   
3        100000158       2020-01          1                0                0   
4        100000158       2020-02          1                0                0   
5        100000158       2020-03          1                0                0   
6        100000158       2020-04          1                0                0   
7        100000158       2020-05          1                0                0   
8        100000158       2020-06          1                0                0   
9        100000158       2020-07          1                0                0   
10       100000158       2020-08          1                0                0   
11       100000158       202

### build the balance run down logic

In [0]:


df_forecast_28['adj_PD'] = df_forecast_28['PD']
df_forecast_28['adj_PA'] = df_forecast_28['PA']

df_forecast_28['POD'] = df_forecast_28['adj_PD'] + df_forecast_28['adj_PA']
df_forecast_28['cum_POD'] = df_forecast_28.groupby(['AcctID'])['POD'].cumsum()
df_forecast_28['POA'] = 1 - df_forecast_28['cum_POD']

df_forecast_28.loc[df_forecast_28['POA']<= 0, 'POA']  = 0
df_forecast_28.loc[df_forecast_28['POA']<= 0, 'adj_PD']  = 0
df_forecast_28.loc[df_forecast_28['POA']<= 0, 'adj_PA']  = 0

df_forecast_28.loc[(df_forecast_28.groupby(['AcctID'])['POA'].shift(1) < df_forecast_28['PD'] + df_forecast_28['PA']) &\
                   (df_forecast_28.groupby(['AcctID'])['POA'].shift(1) >= df_forecast_28['PD']),\
                   'adj_PD'] = df_forecast_28['PD']
df_forecast_28.loc[(df_forecast_28.groupby(['AcctID'])['POA'].shift(1) < df_forecast_28['PD'] + df_forecast_28['PA']) &\
                   (df_forecast_28.groupby(['AcctID'])['POA'].shift(1) >= df_forecast_28['PD']),\
                   'adj_PA'] = df_forecast_28.groupby(['AcctID'])['POA'].shift(1) - df_forecast_28['adj_PD']
df_forecast_28.loc[(df_forecast_28.groupby(['AcctID'])['POA'].shift(1) < df_forecast_28['PD']),\
                   'adj_PD'] = df_forecast_28.groupby(['AcctID'])['POA'].shift(1)
df_forecast_28.loc[(df_forecast_28.groupby(['AcctID'])['POA'].shift(1) < df_forecast_28['PD']),\
                   'adj_PA'] = 0


print(df_forecast_28.loc[df_forecast_28['AcctID'] == 167102028,\
                         ['AcctID', 'forecast_YrMo', 'PD', 'POA','cum_POD', 'adj_PD', 'adj_PA','PA','LGD_CGD']])




# print(df_forecast_28[['AcctID', 'POD', 'cum_POD', 'POA','adj_PD']])


           AcctID forecast_YrMo        PD       POA   cum_POD    adj_PD  \
256714  167102028       2019-10  0.000632  0.958380  0.041620  0.000632   
256715  167102028       2019-11  0.000842  0.916868  0.083132  0.000842   
256716  167102028       2019-12  0.001075  0.873505  0.126495  0.001075   
256717  167102028       2020-01  0.001050  0.836802  0.163198  0.001050   
256718  167102028       2020-02  0.001088  0.801540  0.198460  0.001088   
256719  167102028       2020-03  0.001035  0.762473  0.237527  0.001035   
256720  167102028       2020-04  0.000989  0.723110  0.276890  0.000989   
256721  167102028       2020-05  0.001168  0.675945  0.324055  0.001168   
256722  167102028       2020-06  0.000955  0.637120  0.362880  0.000955   
256723  167102028       2020-07  0.001125  0.597190  0.402810  0.001125   
256724  167102028       2020-08  0.001209  0.558313  0.441687  0.001209   
256725  167102028       2020-09  0.001259  0.523263  0.476737  0.001259   
256726  167102028       2

In [0]:
df_forecast_sum = df_forecast_28.groupby(['forecast_YrMo'])['PD','adj_PD','PA','adj_PA','POA','LGD_CGD','utilization_rate'].apply(lambda x : x.sum())
print(df_forecast_sum)

                      PD     adj_PD          PA      adj_PA           POA  \
forecast_YrMo                                                               
2019-10        55.719756  55.719756  539.136887  539.136887  64019.143357   
2019-11        69.264532  69.264532  491.447824  491.447824  62338.954423   
2019-12        80.589802  80.589802  536.792269  536.792269  60501.604918   
2020-01        73.190835  73.190835  487.840043  487.731778  58073.545604   
2020-02        63.160338  63.157877  492.723871  491.940179  56462.164571   
2020-03        61.936597  61.926165  569.233641  565.898722  54065.458013   
2020-04        56.758576  56.744768  534.826177  529.564385  52292.793383   
2020-05        67.092454  67.072033  510.454501  500.092094  49669.641088   
2020-06        57.800234  57.780092  521.228829  502.379819  47921.851991   
2020-07        60.244016  60.213489  463.737568  443.939302  45148.966796   
2020-08        59.140834  59.103299  450.416662  425.787336  43410.775782   

In [0]:
print(df_forecast_28.loc[(df_forecast_28.groupby(['AcctID'])['POA'].shift(1) < df_forecast_28['PD']),['AcctID','PD','POA','adj_PD']])

            AcctID        PD  POA  adj_PD
10655    101201387  0.000404  0.0     0.0
10656    101201387  0.000440  0.0     0.0
10657    101201387  0.000441  0.0     0.0
10658    101201387  0.000433  0.0     0.0
10659    101201387  0.000449  0.0     0.0
10660    101201387  0.000554  0.0     0.0
10661    101201387  0.000640  0.0     0.0
10662    101201387  0.000582  0.0     0.0
10663    101201387  0.000494  0.0     0.0
10664    101201387  0.000478  0.0     0.0
10665    101201387  0.000427  0.0     0.0
10666    101201387  0.000502  0.0     0.0
10667    101201387  0.000428  0.0     0.0
10668    101201387  0.000460  0.0     0.0
10669    101201387  0.000456  0.0     0.0
10670    101201387  0.000444  0.0     0.0
10671    101201387  0.000457  0.0     0.0
10672    101201387  0.000563  0.0     0.0
10673    101201387  0.000649  0.0     0.0
10674    101201387  0.000589  0.0     0.0
10675    101201387  0.000500  0.0     0.0
10676    101201387  0.000484  0.0     0.0
10677    101201387  0.000432  0.0 

In [0]:
df_forecast_201907 = df_forecast_28.loc[df_forecast_28['index'] == '2021-Sep-30',['AcctID','forecast_YrMo','adj_PD','PD','adj_PA','PA','LGD_CGD','POA']]
df_forecast_201907.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\CompareResults_withNaz2021SeptSnapshot.csv')
print(df_forecast_201907)
print(df_forecast_28.forecast_YrMo)

           AcctID forecast_YrMo    adj_PD        PD    adj_PA        PA  \
3111    101201387       2021-09  0.000597  0.000597  0.018773  0.018773   
7668    104401232       2021-09  0.000546  0.000546  0.017512  0.017512   
8312    105103537       2021-09  0.000570  0.000570  0.016698  0.016698   
8471    105301178       2021-09  0.000546  0.000546  0.033660  0.033660   
8820    106003697       2021-09  0.000000  0.000574  0.000000  0.042186   
9823    106801386       2021-09  0.000541  0.000541  0.024636  0.024636   
10755   107502672       2021-09  0.000540  0.000540  0.027119  0.027119   
12263    11000517       2021-09  0.001082  0.001082  0.024301  0.024301   
13824   110801019       2021-09  0.000554  0.000554  0.031451  0.031451   
18648   112215152       2021-09  0.000511  0.000511  0.024640  0.024640   
19799   112401438       2021-09  0.000531  0.000531  0.020324  0.020324   
20819   112515045       2021-09  0.000511  0.000511  0.024640  0.024640   
20886   112516275       2

In [0]:
# df_forecast_sum = df_forecast_28.groupby(['forecast_YrMo'])['pd_vintage_component','pd_age_component','pd_time_component','pa_vint_component','pd_ssn_component','pa_age_component','pa_time_component','pa_ssn_component','PD','adj_PD','PA','adj_PA','POA','LGD_CGD','gspTX_12m_LogRatio_7m_change_11m_lag', 'gspTX_22m_LogRatio_3m_lag','byAAA_9m_lag','utilization_rate','Revolve_1','IntRt_flag_GE15'].apply(lambda x : x.sum())
df_forecast_sum.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\forecast_summary_2019-9-23.csv')

In [0]:
print(df_forecast_sum['LGD_CGD'])

forecast_YrMo
2019-07    5.198815e-14
2019-08    2.733558e-14
2019-09    5.562469e-14
2019-10    2.933380e-13
2019-11    1.327572e-12
2019-12    3.374490e-12
2020-01    5.617425e-12
2020-02    1.185537e-11
2020-03    3.811666e-11
2020-04    1.019173e-09
2020-05    2.261722e-09
2020-06    4.362235e-10
2020-07    1.654557e-11
2020-08    1.053907e-12
2020-09    2.990453e-13
2020-10    5.465914e-13
2020-11    2.683166e-12
2020-12    3.598342e-11
2021-01    6.194642e-10
2021-02    1.126128e-08
2021-03    1.028833e-07
2021-04    6.911931e-07
2021-05    1.654491e-06
2021-06    3.905449e-06
2021-07    5.114378e-07
2021-08    2.355734e-08
2021-09    1.788022e-08
2021-10    1.162800e-08
2021-11    8.046961e-09
2021-12    6.715951e-09
Freq: M, Name: LGD_CGD, dtype: float64


#### construct the balances for the non-revolving loans

In [0]:
# df_forecast_281 = df_forecast_28.loc[df_forecast_28['AcctID'].isin([113011589, 175701758]) ].reset_index()
#
df_forecast_28['Default_ind_90d'] = 0
df_forecast_28.loc[(df_forecast_28['DAYS_PAST_DUE_CNT'] >= 90) | (df_forecast_28['EOMChrgOffDt'].notnull()) | (df_forecast_28['RiskCd'] >=12), 'Default_ind_90d'] = 1
df_forecast_28['Default_ind_90d'] = df_forecast_28['Default_ind_90d'].astype(int)
df_forecast_281 = df_forecast_28.loc[df_forecast_28.Revolve_1 ==0, ['AcctID','forecast_YrMo','PrinPayment_L6mon',\
                                                                    'adj_PA','adj_PD','UNPAID_BALANCE_AMT','Revolve_1','Default_ind_90d']]
print(df_forecast_28.loc[df_forecast_28['AcctID'] == 100000170,])
print(len(df_forecast_281['AcctID'].unique()))

        index     AcctID     INSTID ProdCd_YrMo_x EOMOpenDt_YrMo OpenDt_str  \
0  2019-10-31  100000170  100000170           PLU      31OCT2011     201110   
1  2019-11-30  100000170  100000170           PLU      31OCT2011     201110   
2  2019-12-31  100000170  100000170           PLU      31OCT2011     201110   
3  2020-01-31  100000170  100000170           PLU      31OCT2011     201110   
4  2020-02-29  100000170  100000170           PLU      31OCT2011     201110   
5  2020-03-31  100000170  100000170           PLU      31OCT2011     201110   
6  2020-04-30  100000170  100000170           PLU      31OCT2011     201110   
7  2020-05-31  100000170  100000170           PLU      31OCT2011     201110   
8  2020-06-30  100000170  100000170           PLU      31OCT2011     201110   
9  2020-07-31  100000170  100000170           PLU      31OCT2011     201110   
10 2020-08-31  100000170  100000170           PLU      31OCT2011     201110   
11 2020-09-30  100000170  100000170           PLU   

In [0]:
%%time

df_forecast_281['count'] = 1
df_forecast_281['periods'] = df_forecast_281.groupby(['AcctID'])['count'].cumsum()
df_forecast_281['principal_pay'] = np.maximum(0, df_forecast_281['PrinPayment_L6mon'] * \
                                                  (1-df_forecast_281['adj_PA']-df_forecast_281['adj_PD']))

df_forecast_281.loc[df_forecast_281['periods'] == 1, 'last_period_balance'] = df_forecast_281['UNPAID_BALANCE_AMT']
df_forecast_281.loc[df_forecast_281['periods'] == 1, 'beg_balance'] =df_forecast_281['UNPAID_BALANCE_AMT']
# np.maximum(0,(df_forecast_281['UNPAID_BALANCE_AMT'] - df_forecast_281['principal_pay']) *\
# (1-df_forecast_281['adj_PA']-df_forecast_281['adj_PD'] ))

df_forecast_281.loc[df_forecast_281['periods'] == 1, 'POA'] = (1-df_forecast_281['adj_PA']-df_forecast_281['adj_PD'] )

acctid_list = df_forecast_281['AcctID'].unique()
# print(len(acctid_list))
# acctid_list = [113011589, 175701758]
# acctid_list = [124702455]

k = pd.DataFrame()
for id in acctid_list:
    g = df_forecast_281.loc[df_forecast_281.AcctID == id, ].reset_index(drop = True)
#     print(id)
    for j in range(1,len(g)):
        balance = g.loc[j -1 ,'last_period_balance'] 
        g.loc[j,'POA'] = g.loc[j-1, 'POA'] - g.loc[j, 'adj_PD'] - g.loc[j, 'adj_PA']
        g.loc[j,'principal_pay'] = np.maximum(0, g.loc[j, 'PrinPayment_L6mon'] * g.loc[j,'POA'])
        g.loc[j,'beg_balance'] = np.maximum(0,balance * g.loc[j-1,'POA'] - g.loc[j-1,'principal_pay'])
        g.loc[j, 'last_period_balance'] = g.loc[j, 'beg_balance']
#         g.loc[j, 'last_period_balance'] = balance

    k = k.append(g, ignore_index = True)

# print(k)


Wall time: 10min 54s


In [0]:
k.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\all_nonRevov_2019-10-17.csv')

KeyboardInterrupt: 

In [0]:
print(k.loc[k.AcctID == 100000170,])
# print (k)
# ['AcctID','forecast_YrMo','end_balance','last_period_balance','principal_pay']

Empty DataFrame
Columns: [AcctID, forecast_YrMo, PrinPayment_L6mon, adj_PA, adj_PD, UNPAID_BALANCE_AMT, Revolve_1, Default_ind_90d, count, periods, principal_pay, last_period_balance, beg_balance, POA]
Index: []


In [0]:
print(df_forecast_28.columns[df_forecast_28.columns.str.contains('Open')])
# print(df_forecast_28[['AcctID','forecast_YrMo']])
print(data_snapshot.columns[data_snapshot.columns.str.contains('Open')])

Index(['EOMOpenDt_YrMo', 'OpenDt_str', 'OpenAmt_YrMo', 'OpenDt_Dt',
       'OpenDt_YrMo', 'OpenAmt_YrMo'],
      dtype='object')
Index(['EOMOpenDt_YrMo', 'OpenAmt_Origin', 'OpenAmt_YrMo', 'EOMOpenDt_Min',
       'OpenDt_Dt', 'OpenDt_YrMo', 'OpenDt_str'],
      dtype='object')


In [0]:
# get additional variables that will be used in the definition for default
# default_data = data_snapshot_1[['INSTID', 'DAYS_PAST_DUE_CNT', 'EOMChrgOffDt', 'RiskCd','OpenAmt_YrMo']]
# k1 = pd.merge(k, default_data, left_on = 'AcctID', right_on = 'INSTID', how ='left')
# print(data_snapshot_1.columns)
# print (default_data)

In [0]:

# Determine EAD
data_forecast_281 = k
data_forecast_281['EAD'] = data_forecast_281['last_period_balance']

# Define default loans and default loans are treated differently and the adj_PD will be set to 1 for default loans
# data_forecast_281['Default_ind_90d'] = 0
# data_forecast_281.loc[(data_forecast_281['DAYS_PAST_DUE_CNT'] >= 90) | (data_forecast_281['EOMChrgOffDt'].notnull()) | (data_forecast_281['RiskCd'] >=12), 'Default_ind_90d'] = 1
# data_forecast_281['Default_ind_90d'] = data_forecast_281['Default_ind_90d'].astype(int)
# data_forecast_281['Default_ind_90d'] = (data_forecast_281["DAYS_PAST_DUE_CNT"] >= 90) | (data_forecast_281["EOMChrgOffDt"].notnull()) | (data_forecast_281['RiskCd'] >=12) 

df_forecast_282 = df_forecast_28.loc[df_forecast_28.Revolve_1 == 1, ]

df_ccf = pd.read_excel("W:\\Corporate\\20368\\DFAST_Enterprise_Credit_Loss_Stress_Testing_Models\\Consumer Models\\SAS_MIP\\ConsumerOther\\Documentation\\ConsumerOther_Models_forSAS_MIP_20190521.xlsx",
                           sheetname='Utilization_table')
df_ccf.rename(columns = {'Utilization_Rate':'ccf_EAD'}, inplace = True)
print(df_ccf.ccf_EAD.dtype)

df_forecast_283 = pd.merge(df_forecast_282, df_ccf,  left_on = 'forecast_mon', right_on = 'Age_mon', how = 'left')

df_forecast_283['ccf_EAD'] = df_forecast_283['ccf_EAD'].fillna(0)
df_forecast_284 = df_forecast_283[['AcctID','forecast_YrMo','PrinPayment_L6mon','OpenAmt_YrMo','adj_PA','adj_PD','UNPAID_BALANCE_AMT','Revolve_1','ccf_EAD','Default_ind_90d']]
#There is one additional column with 'OpenAmt_YrMo'
# df_forecast_284.drop([4], axis = 1)


df_forecast_284['EAD'] =df_forecast_284['OpenAmt_YrMo'] * df_forecast_284['ccf_EAD']
df_forecast_284 = df_forecast_284[['AcctID', 'forecast_YrMo', 'EAD','ccf_EAD']]
df_forecast_281 = data_forecast_281[['AcctID', 'forecast_YrMo', 'EAD']]


df_forecast_29 = df_forecast_281.append(df_forecast_284, ignore_index = True)
df_forecast_29.sort_values(by=['AcctID','forecast_YrMo'], inplace=True)



df_forecast_30 = pd.merge(df_forecast_28, df_forecast_29, on = ['AcctID', 'forecast_YrMo'], how = 'left')

df_forecast_sum = df_forecast_30.groupby(['forecast_YrMo'])['PD','adj_PD','PA','adj_PA','POA','LGD_CGD','EAD'].apply(lambda x : x.sum())
# print(df_forecast_sum)


float64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
# print(df_forecast_30.loc[df_forecast_30.AcctID == 124905460,['AcctID','forecast_YrMo','Revolve_1','ccf_EAD','POA','last_period_balance','EAD','Default_ind_90d']])
# print(df_forecast_30.columns)
# print(df_forecast_282[['AcctID', 'forecast_mon', 'Age']])

In [0]:
# default = df_forecast_30.loc[df_forecast_30['Default_ind_90d'] == 1, ]
# print(default)

In [0]:
# df_forecast_sum.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\summary_forecast_EAD.csv')

In [0]:
# print(list(df_forecast_30.columns[df_forecast_30.columns.str.contains('Reco')]))

In [0]:
print(df_forecast_30[['AcctID', 'forecast_YrMo','OpenAmt_YrMo','ccf_EAD','EAD','Default_ind_90d']])

           AcctID forecast_YrMo  OpenAmt_YrMo   ccf_EAD            EAD  \
0       100000170       2019-10       15000.0  0.405975    6089.626050   
1       100000170       2019-11       15000.0  0.401570    6023.553690   
2       100000170       2019-12       15000.0  0.405733    6085.992285   
3       100000170       2020-01       15000.0  0.407117    6106.757850   
4       100000170       2020-02       15000.0  0.405343    6080.139060   
5       100000170       2020-03       15000.0  0.406219    6093.280620   
6       100000170       2020-04       15000.0  0.405183    6077.746785   
7       100000170       2020-05       15000.0  0.406071    6091.068195   
8       100000170       2020-06       15000.0  0.407023    6105.340110   
9       100000170       2020-07       15000.0  0.407998    6119.975865   
10      100000170       2020-08       15000.0  0.410290    6154.346100   
11      100000170       2020-09       15000.0  0.411828    6177.421950   
12      100000170       2020-10       

In [0]:
tmp = df_forecast_281[df_forecast_281['AcctID'].isin(acctid_list)]
print(tmp)

Empty DataFrame
Columns: [AcctID, forecast_YrMo, EAD, Default_ind_90d]
Index: []


In [0]:
%%time
df_forecast_30.loc[df_forecast_30['Default_ind_90d'] == 1,'adj_PD'] = 1
df_forecast_30['Total_Recovery'] = df_forecast_30['adj_PD']*(1-df_forecast_30['LGD_CGD']) + df_forecast_30['adj_PD']* df_forecast_30['LGD_CGD']*df_forecast_30['Recovery']
df_forecast_30['Loss_Amt'] = df_forecast_30['EAD'] * df_forecast_30['adj_PD']*(1 - df_forecast_30['Total_Recovery'])
# df_forecast_30.loc[df_forecast_30['index'] == '2019-07-31','Prev_Total_Loss_Amt'] = df_forecast_30['Loss_Amt']
df_forecast_30.loc[df_forecast_30['index'] == '2019-10-30','total_loss_amt'] = df_forecast_30['Loss_Amt']

df_forecast_31 = df_forecast_30[['AcctID','index','forecast_YrMo','EAD','Total_Recovery','Loss_Amt','total_loss_amt','adj_PD','adj_PA','LGD_CGD','segment_level_s',]]
# acctid_list = [113011589, 175701758]
acctid_list = df_forecast_31['AcctID'].unique()
# acctid_list = list(acctid_list_0)[0:3]
# print(df_forecast_31)


l = pd.DataFrame()
for id in acctid_list:
    g = df_forecast_31.loc[df_forecast_31.AcctID == id, ].reset_index(drop = True)
#     print(id)
    for j in range(1,len(g)):
#         prev_loss = g.loc[j-1 ,'total_loss_amt'] 
        g.loc[j, 'total_loss_amt'] = g.loc[j, 'Loss_Amt'] + g.loc[j-1, 'total_loss_amt']
#         print(g.loc[j, 'Loss_Amt'], g.loc[j, 'EAD'])
#         print(g.loc[j, 'adj_PD'])
#         g.loc[j, 'Prev_Total_Loss_Amt'] = g.loc[j, 'total_loss_amt']

    l = l.append(g, ignore_index = True)



Wall time: 27min 23s


In [0]:
print(l)

           AcctID      index forecast_YrMo            EAD  Total_Recovery  \
0       100000170 2019-07-31       2019-07    6225.718695        0.000480   
1       100000170 2019-08-31       2019-08    6181.416990        0.000488   
2       100000170 2019-09-30       2019-09    6146.230230        0.000456   
3       100000170 2019-10-31       2019-10    6089.626050        0.000487   
4       100000245 2019-07-31       2019-07  134890.571725        0.000480   
5       100000245 2019-08-31       2019-08  133930.701450        0.000488   
6       100000245 2019-09-30       2019-09  133168.321650        0.000456   
7       100000245 2019-10-31       2019-10  131941.897750        0.000487   
8       100000245 2019-11-30       2019-11  130510.329950        0.000586   
9       100000245 2019-12-31       2019-12  131863.166175        0.000689   
10      100000245 2020-01-31       2020-01  132313.086750        0.000641   
11      100000245 2020-02-29       2020-02  131736.346300        0.000561   

In [0]:
loan2 = l
# loan2.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\two_loan_loss_snapshot.csv')


In [0]:
loan2_summary = loan2.groupby(['forecast_YrMo', 'segment_level_s'])['Loss_Amt','total_loss_amt','EAD'].apply(lambda x : x.sum())
loan2_summary.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\summary_Loss_bySegments_201910.csv')

In [0]:
ecl_result = l


In [0]:
ecl_snapshot = ecl_result.loc[ecl_result['index'] == '2019-08-31',]
ecl_snapshot.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\ecl_snapshot_201908.csv')

In [0]:
ecl_summary = ecl_result.groupby(['forecast_YrMo'])['Loss_Amt','total_loss_amt'].apply(lambda x : x.sum())
ecl_summary.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\summary_Loss.csv')
print(ecl_summary)

                    Loss_Amt  total_loss_amt
forecast_YrMo                               
2019-07        491221.032216    4.912210e+05
2019-08        467684.224516    9.410500e+05
2019-09        420224.362580    1.295802e+06
2019-10        404006.921762    1.615409e+06
2019-11        461215.667773    1.970257e+06
2019-12        498696.319633    2.368525e+06
2020-01        420699.428205    2.646739e+06
2020-02        333859.183111    2.828354e+06
2020-03        294480.734484    2.868386e+06
2020-04        241126.588627    2.860442e+06
2020-05        248685.054367    2.725611e+06
2020-06        189851.995231    2.638590e+06
2020-07        160081.705345    2.296121e+06
2020-08        143262.976314    2.238915e+06
2020-09        126107.536032    2.162726e+06
2020-10        118052.320435    2.085798e+06
2020-11        127534.336650    1.984719e+06
2020-12        127319.653188    1.897248e+06
2021-01         97050.264425    1.778308e+06
2021-02         69984.666511    1.656991e+06
2021-03   

In [0]:
df_forecast_201907 = df_forecast_30.loc[df_forecast_30['index'] == '2019-Sep-30',['AcctID','forecast_YrMo','adj_PD','adj_PA','ccf_EAD','EAD','Total_Recovery','Loss_Amt']]
df_forecast_201907.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\CompareResults_withNaz201909EADSnapshot.csv')

In [0]:
df_forecast_201907 = df_forecast_28.loc[df_forecast_28['forecast_YrMo'] == '2019-07',['AcctID','forecast_YrMo','adj_PA','adj_PD','Prev_Total_Loss_Amt']]
df_forecast_201907.to_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\CompareResults_withNaz2021SeptSnapshot.csv')

In [0]:
check_tmp = df_forecast_30.loc[df_forecast_30['AcctID'] == 124702455, ['AcctID','forecast_YrMo','adj_PD','adj_PA','ccf_EAD','EAD','UNPAID_BALANCE_AMT','principal_payment','PrinPayment_L6mon']]
print(check_tmp)

          AcctID forecast_YrMo    adj_PD    adj_PA  ccf_EAD            EAD  \
50300  124702455       2019-07  0.000783  0.033901      NaN  641036.840000   
50301  124702455       2019-08  0.000709  0.033862      NaN  471826.489978   
50302  124702455       2019-09  0.000652  0.029682      NaN  471864.351332   
50303  124702455       2019-10  0.000661  0.029516      NaN  309209.638062   

       UNPAID_BALANCE_AMT  principal_payment  PrinPayment_L6mon  
50300           641036.84                NaN          157728.28  
50301           641036.84                NaN          157728.28  
50302           641036.84                NaN          157728.28  
50303           641036.84                NaN          157728.28  


In [0]:
df_forecast_28['last_period_balance'] = df_forecast_28.groupby(['AcctID'])['UNPAID_BALANCE_AMT'].shift(1)
print(df_forecast_28[['AcctID', 'forecast_YrMo', 'UNPAID_BALANCE_AMT', 'last_period_balance']])

           AcctID forecast_YrMo  UNPAID_BALANCE_AMT  last_period_balance
0       100000170       2019-07                0.00                  NaN
1       100000170       2019-08                0.00                 0.00
2       100000170       2019-09                0.00                 0.00
3       100000170       2019-10                0.00                 0.00
4       100000245       2019-07           162598.12                  NaN
5       100000245       2019-08           162598.12            162598.12
6       100000245       2019-09           162598.12            162598.12
7       100000245       2019-10           162598.12            162598.12
8       100000245       2019-11           162598.12            162598.12
9       100000245       2019-12           162598.12            162598.12
10      100000245       2020-01           162598.12            162598.12
11      100000245       2020-02           162598.12            162598.12
12      100000245       2020-03           162598.12

In [0]:
# df_forecast_28.loc[df_forecast_28['RevolveIn'] == 1, 'current_balance'] = df_forecast_28['OpenAmt_YrMo'] * df_forecast_28['Utilization_Rate']
# df_forecast_28['Revolve_1'] =  df_forecast_28['Revolve_1'].astype(int)
df_forecast_28.loc[df_forecast_28['Revolve_1'] == 1, 'current_balance'] = df_forecast_28['OpenAmt_YrMo'] * df_forecast_28['Utilization_Rate']

TypeError: '<' not supported between instances of 'str' and 'int'

In [0]:
# 

In [0]:
print(list(df_forecast_28))

['index', 'AcctID', 'INSTID', 'ProdCd_YrMo_x', 'EOMOpenDt_YrMo', 'OpenDt_str', 'EOMMaturityDt_YrMo', 'MaturityDate_mon', 'OpenAmt_YrMo', 'Term_Origin', 'EFFECTIVE_INTEREST_RT', 'RevolveIn', 'segment_level_0', 'segment_level_1', 'vintage_year', 'IntRt_flag_GE15', 'utilization_rate', 'PMT_L6MO', 'Age_mon', 'OpenDt_Dt', 'OpenDt_YrMo', 'Vintage_Days', 'MaturityDate_dt', 'NetGLOutstandingBalance', 'OpenAmt_YrMo', 'PrinPayment_L6mon', 'forecast_YrMo', 'forecast_mon', 'ssn_mon', 'Date', 'gspTX', 'itbyBAA', 'byAAA', 'gspTX_22m_LogRatio_3m_lag', 'gspTX_12m_LogRatio_7m_change_11m_lag', 'itbyBAA_24m_Diff_12m_lag', 'byAAA_9m_lag', 'pd_age_component', 'pd_age_lookup_key', 'pd_ssn_lookup_key', 'pd_ssn_component', 'pd_vint_lookup_key', 'pd_vintage_component', 'age_seg_key', 'vint_seg_key', 'ssn_seg_key', 'forecast_YrMo_1', 'time_seg_key', 'pa_age_component', '_name__x', 'pa_ssn_component', '_name__y', '_name__x', 'pa_vint_component', 'pa_time_component', '_name__y', 'PA', 'pd_time_component', 'PD', '

#### read in the PrinPayment file prepared by Brennan

In [0]:
Prin_Payment = pd.read_csv('C:\\Users\\Hu09559\\Documents\\Consumer\\Reserve Forecast\\PORTFOLIO_FLAT_PrinPayment_L6mon.csv')

In [0]:
# Prin_all = pd.merge(Prin_Payment, data_snapshot, on = 'INSTID', how = 'left')
print(list(Prin_all.columns[Prin_all.columns.str.contains('Rev')]))

['RevolveIn', 'CI_NonRev_Ind', 'CI_Rev_Ind']


In [0]:
print(list(Prin_all.columns))

['REPORTING_DT_x', 'YrMo_x', 'INSTID', 'NetGLOutstandingBalance_x', 'PrinPayment_L6mon', '_batch_var_', 'REPORTING_DT_y', 'ASOFDATE', 'YrMo_y', 'ENTITY_ID', 'WORKGROUP', 'PRODUCT_TYPE_DESC', 'PORTFOLIO_SEGMENT', 'OLD_PRODUCT_TYPE_DESC_TEMP', 'OLD_PRODUCT_TYPE_DESC', 'QFACTOR_PRODUCT', 'ACCOUNTING_METHOD', 'ASSET_TYPE_DESC', 'INSTTYPE', 'LINE_OF_BUSINESS_DESC', 'CURRENCY', 'SEGMENT_ID', 'PD_SEGMENT_ID', 'START_DT', 'SCHEDULE_START_DT', 'MATURITY_DT', 'PRINCIPAL_PAYMENT_DT', 'INTEREST_PAYMENT_DT', 'NEXT_RESET_DT', 'UNPAID_BALANCE_AMT', 'CURRENT_RT', 'EFFECTIVE_INTEREST_RT', 'MARGIN_RT', 'INDEX_REFERENCE', 'PAYMENT_AMT', 'DAYS_PAST_DUE_CNT', 'MODIFICATION_DT', 'ASSESSMENT_TYPE_DESC', 'COLLATERAL_DESC', 'PENDING_OFFER_FLG', 'DRAWDOWN_PERIOD_FLG', 'ISSUED_GUARANTEE_FLG', 'REVOLVING_FLG', 'OFF_BALANCE_AMT', 'CREDIT_LIMIT_AMT', 'DELINQUENCY_FLG', 'MOST_RECENT_DELINQUENCY_DT', 'SECTOR_DESC', 'COUNTERPARTY_TYPE', 'LOAN_STATUS_DESC', 'GEOGRAPHY_CD', 'CCF', 'INTEREST_RATE_TYPE', 'INTEREST_PAYMENT

In [0]:
Prin_all_0 = Prin_all[['INSTID', 'NetGLOutstandingBalance_x', 'PrinPayment_L6mon', 'RevolveIn', 'REVOLVING_FLG','PrinPayments']]
print(Prin_all_0.head())

      INSTID  NetGLOutstandingBalance_x  PrinPayment_L6mon  RevolveIn  \
0  100000158                       0.00               6.00        NaN   
1  100000169                       0.00               3.00        NaN   
2  100000170                       0.00               3.00        1.0   
3  100000183                   40000.00          -19997.00        NaN   
4  100000245                  162598.12          -61296.06        1.0   

  REVOLVING_FLG  PrinPayments  
0           NaN           NaN  
1           NaN           NaN  
2             Y           0.0  
3           NaN           NaN  
4             Y        3225.4  


In [0]:
prin_miss_cnt = Prin_all_0['PrinPayments'].isnull().sum()
print(prin_miss_cnt)
revolving_cnt = Prin_all_0[Prin_all_0['RevolveIn'] == 1, ].shape(0)

37805


In [0]:
Prin_neg = Prin_all_0.loc[Prin_all_0['REVOLVING_FLG'] == 'Y',]

In [0]:
print(Prin_neg)

          INSTID  NetGLOutstandingBalance_x  PrinPayment_L6mon  RevolveIn  \
2      100000170                       0.00              3.000        1.0   
4      100000245                  162598.12         -61296.060        1.0   
6      100001161                       0.00          46201.075        1.0   
7      100001250                    4748.54          -2371.270        1.0   
9       10000128                       0.00          48314.515        1.0   
10     100001282                    4904.81          -2449.405        1.0   
11     100001326                       0.00           2455.405        1.0   
14     100002407                   35000.00          50068.850        1.0   
15     100002422                       0.00          17503.000        1.0   
18     100003237                    2513.72          14254.175        1.0   
21       1000114                    4905.57          -1049.730        1.0   
22       1000127                       0.00           2455.785        1.0   

In [0]:
for sheet in df_lookups.sheets():
    print (sheet.name)

forecast_logic_NonRev
Models
LGD_recovery_table
Utilization_table
PA_age
PA_ssn
PA_vintage
PA_time
PD_Vintage
PD_Age
PD_Ssn
logic for defining two-tier seg
